In [105]:
#!pip install --user -r requirements.txt

In [3]:
import warnings
warnings.filterwarnings("ignore")
import random
import time
import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import os

# Set up TWINT config
import twint
# Solve compatibility issues with notebooks and RunTime errors.
import nest_asyncio

# stopword
import nltk
from nltk.corpus import stopwords
nltk.download("vader_lexicon")
nltk.download('stopwords')
sw_nltk = stopwords.words('english')

import re
text_cleaning_regex = "@S+|https?:S+|http?:S|[^A-Za-z0-9]+"

# sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support,roc_curve, auc, f1_score,cohen_kappa_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# bert
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler,random_split
from transformers import BertModel,BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AdamW, get_linear_schedule_with_warmup


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/swu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/swu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# check the device
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [5]:
def tweets_api(keyword, Language, min_replies):
    '''
    :param keyword:
    :param Language:
    :param min_replies:
    :return: processed dataframe
    '''
    nest_asyncio.apply()
    c = twint.Config()
    c.Search = keyword
    c.Lang = Language
    c.Store_pandas = True
    # c.Store_csv = True
    # c.Output = "test.csv"
    c.Since = '2019-03-16 14:00:00'
    c.until = '2022-03-01 06:00:00'
    c.Min_replies = min_replies
    c.Pandas = True
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    df = df[["date", "tweet", "nreplies", "nretweets", "nlikes"]]
    return df

In [6]:
df = tweets_api(keyword = 'BTC', Language = 'en', min_replies = 5)

1503216234698997766 2022-03-13 20:47:33 -0700 <luckychartape> I am actually Bullish on $eth. I have been studying the eth charts for a while, and the eth/BTC pairing. I think there is a significant change we breakout soon. I may be completely wrong. This is a pretty bold call given the market  https://t.co/84KK0UuxNO
1503215014626439172 2022-03-13 20:42:42 -0700 <TidalGlobal> If you're looking for a reason to get into Tidal Global, there's no better one than this!   https://t.co/NcSeekhkgm  #matic #polygon #dapp #eth #bnb #btc #productivity #nft #data #disruption  https://t.co/Ke7b4RlkAZ
1503214128663613443 2022-03-13 20:39:11 -0700 <easyeight08> DAMN, #BTC REALLY LOOKS LIKE SHIT. IT CLOSED LIKE SHIT ON ALL TIMEFRAMES. IT's EITHER BEARRAP OR ALL THESE STUPID BEARS WERE RIGHT AND DUMP WILL BE MASSIVE.
1503209908753551361 2022-03-13 20:22:25 -0700 <DegenSpartan> ~5 years ago   ETH was $79  BTC was $1347
1503208661241450498 2022-03-13 20:17:27 -0700 <mewn21> i just realized how easy it is

1503180976758874112 2022-03-13 18:27:27 -0700 <DarrenPromoter> FREE #NFT GIVEAWAY!! 🎉  RT + Follow @DarrenPromoter &amp; drop your $ETH wallet! 💵  #NFTGiveaway #NFTGiveaways #NFTCommmunity #picknava #NFTs #ETH #BTC  #Crypto #BNB  #SOL #BSC #Avax  #Doge #Ethereum #Bitcoin  #Solana #Giveaway #GiveawayNFT #Airdrop #NFTArt #NFTdrop #DropyourNFT  https://t.co/bdyG6frcC8
1503180958933348352 2022-03-13 18:27:23 -0700 <SaitaInformer> #IMPACTXP recovering nicely after the #BTC dump   Dubai will see huge announcements for the project  - Mining facilities generate passive income  - P2E game fully revealed  - Natural energy renewables to generate income and sell excess energy  - Major partnerships  - Much more🔥
1503180822861467656 2022-03-13 18:26:50 -0700 <Amin62874656> @cz_binance $BTC is future.  https://t.co/l846tg0Dmx
1503180446032617473 2022-03-13 18:25:20 -0700 <DarrenPromoter> FREE #NFT GIVEAWAY!! 🎉  RT + Follow @DarrenPromoter &amp; drop your $ETH wallet! 💵  #NFTGiveaway #NFTGiveaways #NF

1503165881341431812 2022-03-13 17:27:28 -0700 <DarrenPromoter> FREE #NFT GIVEAWAY!! 🎉  RT + Follow @DarrenPromoter &amp; drop your $ETH wallet! 💵  #NFTGiveaway #NFTGiveaways #NFTCommmunity #picknava #NFTs #ETH #BTC #Crypto #BNB #SOL #BSC #Avax #Doge #Ethereum #Bitcoin #Solana #Giveaway #GiveawayNFT #Airdrop #NFTArt #NFTdrop #DropyourNFT  https://t.co/BK4xYBXLrY
1503165840455254019 2022-03-13 17:27:18 -0700 <DarrenPromoter> FREE #NFT GIVEAWAY!! 🎉  RT + Follow @DarrenPromoter &amp; drop your $ETH wallet! 💵  #NFTGiveaway #NFTGiveaways #NFTCommmunity #picknava #NFTs #ETH #BTC #Crypto #BNB #SOL #BSC #Avax #Doge #Ethereum #Bitcoin #Solana #Giveaway #GiveawayNFT #Airdrop #NFTArt #NFTdrop #DropyourNFT  https://t.co/eEOj9pk2CT
1503165767709335556 2022-03-13 17:27:01 -0700 <DarrenPromoter> FREE #NFT GIVEAWAY!! 🎉  RT + Follow @DarrenPromoter &amp; drop your $ETH wallet! 💵  #NFTGiveaway #NFTGiveaways #NFTCommmunity #picknava #NFTs #ETH #BTC #Crypto #BNB #SOL #BSC #Avax #Doge #Ethereum #Bitcoin #So

1503154006394388482 2022-03-13 16:40:17 -0700 <DarrenPromoter> FREE #NFT GIVEAWAY!! 🎉  RT + drop your $ETH wallet &amp; RT my pinned tweet! 📌  #NFTGiveaway #NFTGiveaways #NFTCommmunity #picknava #NFTs #ETH #BTC #Crypto #BNB #SOL #BSC #Avax #Doge #Ethereum #Bitcoin #Solana #Giveaway #GiveawayNFT #Airdrop #NFTArt #NFTdrop #DropyourNFT #Cryptos  https://t.co/Nnq3XkQswO
1503153615485210626 2022-03-13 16:38:43 -0700 <DarrenPromoter> FREE #NFT GIVEAWAY!! 🎉  RT + drop your $ETH wallet &amp; RT my pinned tweet! 📌  #NFTGiveaway #NFTGiveaways #NFTCommmunity #picknava #NFTs #ETH #BTC #Crypto #BNB #SOL #BSC #Avax #Doge #Ethereum #Bitcoin #Solana #Giveaway #GiveawayNFT #Airdrop #NFTArt #NFTdrop #DropyourNFT #Cryptos  https://t.co/CvrRbMKYDg
1503153176937074690 2022-03-13 16:36:59 -0700 <DarrenPromoter> FREE #NFT GIVEAWAY!! 🎉  RT + drop your $ETH wallet &amp; RT my pinned tweet! 📌  #NFTGiveaway #NFTGiveaways #NFTCommmunity #picknava #NFTs #ETH #BTC #Crypto #BNB #SOL #BSC #Avax #Doge #Ethereum #Bitco

1503146193370722319 2022-03-13 16:09:14 -0700 <ladymarketok> 3 de 3 - Lady whale  $MASK $REN #BTC  https://t.co/W7XAsrKPHJ
1503145969130651651 2022-03-13 16:08:20 -0700 <Emijaop> #BTC  اینجا چیکار میکنی اهل فن نظر بدند؟ تایم فریم یک دقیقه  https://t.co/BhO9QW0v1A
1503145770630979591 2022-03-13 16:07:33 -0700 <qadqaz> When its time $BTC won’t give a dam about your fancy charts or indicators its just gonna do its thing. Every big pump/dump always happened totally against any TA. Don’t try to catch a perfect bottom here just leave it alone until its done shitting.  #Bitcoin #Crypto
1503145723973423107 2022-03-13 16:07:22 -0700 <ZeussCapital> Few more hours Zeuss fam  BLOW THIS UP 😤⚡️  Enter 👇🏼 #NFTGiveaway #Giveaway $ETH #ETH $BTC #BTC
1503145699101356033 2022-03-13 16:07:16 -0700 <LeMTempete> @MAROTVSHOW Please dm @MAROTVSHOW I need a logo and a banner il leur faut un metamask trust wallet need help #ETH #BTC  solana coinbase binance account blocked
1503145674849804296 2022-03-13 16:07:1

...

1500181305245454340 2022-03-05 10:47:49 -0700 <MartaVerse> Me if #btc drops any lower. Cope will be real  https://t.co/ko1hZJvCRG
1500180754252197890 2022-03-05 10:45:38 -0700 <JoshuaBarbin1> @saylor Gold is the best monetary technology ever created. Better than #btc
1500180749714137090 2022-03-05 10:45:37 -0700 <EnzoPalma> Eso de minar btc da plata?
1500180726360203267 2022-03-05 10:45:31 -0700 <Taylorcrypto_> The market keeps going down..  This is not the time to feel fear, on the contrary, these are opportunities that cannot be missed 👀🚀  And you, where will you invest? tell me below ⬇️🔥  #BTC #Crypto #Binance #BNB #ETH #NFTs #GameFi #altcoins #shitcoin #Bitcoin #NFT #CryptoNews
1500179597257760772 2022-03-05 10:41:02 -0700 <Ridwan_5Diamond> 2012 you missed #BTC          2013 you missed #DOGE 2014 you missed #XRP 2015 you missed #ETH 2016 you missed #ADA 2017 you missed #BNB          2018 you missed #LINK 2019 you missed #DOT 2021 you missed #ATLAS 2022 don’t miss #?????  What #Altc

1500170813600899072 2022-03-05 10:06:08 -0700 <CryptoPNZ> Bitcoin önceki günlerde iki kere saatlik #RET yemiş. şuanda tekrardan yükselişe geçebilmesi için kırılım gerekiyor. aksi halde tekrar ret yemesi durumunda düşüşlerin devamı geleceği yönde.  #Bitcoin direnç: 39.635$ dolar $BTC Destek: 38.693$ dolar  Bol kazançlar.  https://t.co/O5NFDATqwA
1500170719338147842 2022-03-05 10:05:45 -0700 <banana83029121> @BTC_Archive If they can also Russia can, avoiding all sanctions.
1500170060265967616 2022-03-05 10:03:08 -0700 <Epiwhisper> @BitcoinMagazine Bitcoin is a once in a generation opportunity. I feel late to the game yet we are still in the “Internet in 1995” days with respect to BTC. I believe BTC will have as a profound an impact on society as the Internet has.
1500169823921352716 2022-03-05 10:02:12 -0700 <Hector_eth> Up or Down? #BTC #Bitcoin #Crypto  https://t.co/XYzAsqBPlE
1500169770926329857 2022-03-05 10:01:59 -0700 <KinjaGangNFT> #NytProfilbillede One of my 14 Wolves @cunningWol

1500157938274607112 2022-03-05 09:14:58 -0700 <mdtrade> #BTC following expected #Elliottwave path precisely. imo we will see strong downward pressure on price this coming week once the small correction higher is complete. $Bitcoin #Evai  https://t.co/7hbjqWVpfm
1500157837087027201 2022-03-05 09:14:34 -0700 <ProfesorTurkmen> Herhangi birisinin "sözde teknik analiz" açıklamasında yer alan "#BTC'de 40.000 önemli arkadaşlar 🙏🏼" ne anlama geliyor?
1500157832376762376 2022-03-05 09:14:33 -0700 <JohalMiles> If we look back at Bitcoin's history we can see that 2 consecutive green monthly candles more often than not occurs when exiting a local bottom and leads to more upside.  One anomaly in 2014.  If March closes bullish, we should be watching for continuation to the upside. $BTC  https://t.co/V536sOC2Nm
1500157778278682626 2022-03-05 09:14:20 -0700 <KoinSaati> 5 Mart 2022 CoinGecko verilerine göre Türkiye trendlerine giren 10 proje 👇  #BABYDOGE $RACA $PIT $BCOIN $FLOKI $HAM  $ZIG $AVAX $BTC $

1500147641572855817 2022-03-05 08:34:03 -0700 <cryptolawy3r> 5 Kişiye 1250 TL Değerinde Çekiliş !😁 50x5 #MATIC değerinde toplam 5 kişiye #NFT hediye 😁  1️⃣RT &amp; FAV 2️⃣Takip : @ThePenguinSqua1  3️⃣2 kişi etiketle   30H ⏱📢  #NFTGiveaway #Cekilis #eth #btc  https://t.co/U80E4YTAyY
1500147580432531458 2022-03-05 08:33:49 -0700 <DYORToken_BSC> Reminder: no 18:00 GMT #Crypto Coffee.   Join us at 02:00 GMT or 9pm EST/8pm CST to announce Twitter WINNERS, announcements of upcoming AMA's, and more!!! Exciting times for #DYOR Project!!!!  @DYOR_jetrich @Striking_Balls @DyorArmy @DBT_Official @rugby_uc  #BTC #BSC #Binance  https://t.co/fbvpoV68xe
1500146817291411461 2022-03-05 08:30:47 -0700 <TheVladCostea> The philosophical difference between Bitcoin and Etherium can be easily observed in the names of two popular projects:  BTC – @selfbankt  ETH – @BanklessHQ   One former is about empowering the individual, the latter promotes replacing banks with another trusted third party.
1500146340654989

1500093773325352961 2022-03-05 05:00:00 -0700 <Investingcom> 📈📉WALL STREET WEEKLY SUMMARY:  *DOW: -1.3%, 4th WEEKLY LOSS IN A ROW *S&amp;P 500: -1.3% *NASDAQ: -2.8% *RUSSEL 2000: -1.9%  *VIX: +15.9%  *OIL: +22.4% *GOLD: +4.2% *BITCOIN: -0.1%  🇺🇸🇺🇸 $DIA $SPY$QQQ $IWM $VIX $CL_F $GC_F #BTC  https://t.co/lEqKo4baEW
1500093315269672963 2022-03-05 04:58:11 -0700 <CryptoPoseidonn> $BTC  It may be the last opportunity to buy #Bitcoin under $37k.  https://t.co/QhY4kgoXty
1500091814107181056 2022-03-05 04:52:13 -0700 <Degenomics2000> What gem are you loading up on right now??   #btc #ALTSEASON #bnb #1000xgem #BSCGemsAlert
1500090091754115075 2022-03-05 04:45:22 -0700 <Phoenix_Ash3s> First trade after a week long break. Of course it’s underwater! $BTC  https://t.co/gRWKMWpGND
1500089746017689601 2022-03-05 04:44:00 -0700 <YourAnonNews> Heads up, if crypto pal thinks they're on Facebook with a Russian in Russia today - at best - they probably aren't ... YAN #Helping   (╭ರ_•́ ⁄ ¯  https://t.co/GKX

In [8]:
# Transform result to pandas
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]
df = twint_to_pandas(["date", "tweet", "nreplies", "nretweets","nlikes"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8508 entries, 0 to 8507
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   date       8508 non-null   object
 1   tweet      8508 non-null   object
 2   nreplies   8508 non-null   int64 
 3   nretweets  8508 non-null   int64 
 4   nlikes     8508 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 332.5+ KB


## Sentiment analysis

### 1. Preprocessing

In [9]:
def text_preprocessing(tweet):
    tweet = re.sub(text_cleaning_regex, ' ', str(tweet).lower()).strip()
    res=[]
    # Lowercase
    tweet = tweet.lower()
    # Remove single letter words
    # tweet = ' '.join( [w for w in tweet.split() if len(w)>1] )
    # Remove stopword
    tweet = ' '.join([word for word in tweet.split() if word.lower() not in sw_nltk])
    # remove the word after @ OR #
    for i in tweet.split():
        if i.startswith("@") or i.startswith("#"):
            continue
        else:
            res.append(i)
    return ' '.join(res)

In [10]:
df['tweet']=df['tweet'].apply(lambda x: text_preprocessing(x))

In [11]:
df.head()

,date,tweet,nreplies,nretweets,nlikes
0,2022-03-13 20:47:33,actually bullish eth studying eth charts eth b...,6,0,21
1,2022-03-13 20:42:42,looking reason get tidal global better one htt...,23,144,155
2,2022-03-13 20:39:11,damn btc really looks like shit closed like sh...,14,4,62
3,2022-03-13 20:22:25,5 years ago eth 79 btc 1347,6,1,46
4,2022-03-13 20:17:27,realized easy divide conquer crypto space inhe...,5,2,75


### 2. LRSentiA

**training strategy** 
- Generate Labels using an Unsupervised method
- referenece: https://openreview.net/forum?id=kQns9y_JH6, https://www.sciencedirect.com/science/article/pii/S2666827021000074


> LRSentiA is a lexicon and rule-based method that can classify sentiment without using any labeled data. The main purpose of introducing LRSentiA is to generate accurate pseudo-labels so that supervised ML classifiers can be incorporated into SSentiA.

In [12]:
# df['n_words'] = df['tweet'].apply(lambda x: len(x.split()))

In [13]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
df['sentiment_dict'] = df['tweet'].apply(lambda x:analyzer.polarity_scores(x))
df['compound']  = df['sentiment_dict'].apply(lambda score_dict: score_dict['compound'])
df['neg']  = df['sentiment_dict'].apply(lambda score_dict: score_dict['neg'])*-1
df['pos']  = df['sentiment_dict'].apply(lambda score_dict: score_dict['pos'])

#  Prediction confidence
df['confidence score'] = abs(df['neg']+df['pos'])/(abs(df['neg'])+abs(df['pos']))
df['confidence score'] = df['confidence score'].replace(np.nan, 0)

In [14]:
def vader_sentiment_result(sent):
    scores = analyzer.polarity_scores(sent)
    if scores["compound"] >=0.05: # positive, reference: https://pypi.org/project/vader-sentiment/
        return 2
    elif scores["compound"] <=-0.05: # negative
        return 0
    else:
        return 1 # netural

In [15]:
df["vader_result"] = df["tweet"].apply(lambda x: vader_sentiment_result(x))

In [16]:
# df.groupby('vader_result').agg({"tweet":"nunique"}).reset_index()

In [17]:
# use very-high and high confidence groups of positive and negative predictions as training data
df_train = df[df['confidence score']>0.5]
df_test = df[~(df['confidence score']>0.5)]

In [18]:
X_train = df_train.tweet.values
y_train = df_train.vader_result.values
X_test = df_test.tweet.values
y_test = df_test.vader_result.values

In [19]:
# Create a function to tokenize the input for encoder 
# Load bert tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
def preprocessing_for_bert(X, y, batch_size = 32):
    input_ids = []
    attention_masks = []
    
    for sent in X:
        encoded_sent = tokenizer.encode_plus(
            text=sent,  
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=280,                  # maximum twitter lenght 280
            pad_to_max_length=True,         
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True,      
            truncation=True
            )
        
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))
     
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    
    labels = torch.tensor(y)
    # convert the tensors into a PyTorch Dataset=
    data = TensorDataset(input_ids, attention_masks, labels)
    sampler = RandomSampler(data)
    # feed dataset to training loop
    dataloader = DataLoader(data, # The training samples.
                            sampler=sampler,   # Select batches randomly
                            batch_size=batch_size)
    
    return dataloader

In [20]:
train_dataloader  = preprocessing_for_bert(X_train, y_train, batch_size = 32)
test_dataloader  = preprocessing_for_bert(X_test, y_test, batch_size = 32)

In [273]:
# # create the tensor dataset based on differnet confidence score
# def subset(lower,higher):
#     X = df[(df['confidence score']>lower)&(df['confidence score']<=higher)].tweet.values
#     y = df[(df['confidence score']>lower)&(df['confidence score']<=higher)].vader_result.values
#     return preprocessing_for_bert(X, y, batch_size = 32)

In [274]:
# vh_dataloader = subset(lower=0.75,higher=1)
# h_dataloader = subset(lower=0.55,higher=0.75)
# l_dataloader = subset(lower=0.38,higher=0.55)
# vl_dataloader = subset(lower=0, higher=0.38)

### 3. model training

In [23]:
def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    bert_classifier = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

#     #Tell PyTorch to run the model on GPU
#     bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),lr=1e-5) # change the learning rate to a lower number 

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs
    
    # Warm up steps is a parameter which is used to lower the learning rate in order to reduce the impact of deviating the model from learning on sudden new data set exposure.

    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=50, 
                                                num_training_steps=total_steps)
    
    return bert_classifier, optimizer, scheduler

In [24]:
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [143]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
        
def kappa_score(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return cohen_kappa_score(preds_flat,labels_flat, labels=None, weights=None)

In [29]:
def other_metrics(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    f1_score = f1_score(labels_flat, preds_flat, average='weighted')
    kappa_score = cohen_kappa_score(preds_flat,labels_flat, labels=None, weights=None)
    print(" F1 Score: {0:.2f}".format(f1_score))
    print(" Kappa Score: {0:.2f}".format(kappa_score))

#### Training Loop

In [144]:
# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

def train(model, train_dataloader, epochs):
    """
    Train the BertClassifier model
    """
    train_loss_set = []
    print("Start training...\n")
    
    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []
    
    # Measure the total training time for the whole run.
    total_t0 = time.time()
    
    # For each epoch...
    for epoch_i in range(0, epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        
        t0 = time.time()  # Measure how long the training epoch takes.
        total_train_loss = 0  # Reset the total loss for this epoch.
        model.train()
        
        for step, batch in enumerate(train_dataloader):
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            
            model.zero_grad()   
            
            output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
            loss = output.loss
            logits = output.logits
            
            total_train_loss += loss.item()
            # Perform a backward pass to compute gradients
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # Update the model’s parameters 
            optimizer.step()
            # Update the learning rate
            scheduler.step()
            # Calculate the average loss over all of the batches.
        
        avg_train_loss = total_train_loss / len(train_dataloader) 
        training_time = format_time(time.time() - t0)
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))

In [145]:
train(bert_classifier, train_dataloader, epochs=4)

Start training...


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    112.    Elapsed: 0:34:59.
  Batch    80  of    112.    Elapsed: 1:09:38.

  Average training loss: 0.76
  Training epcoh took: 1:37:12

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    112.    Elapsed: 0:34:32.
  Batch    80  of    112.    Elapsed: 1:08:43.

  Average training loss: 0.41
  Training epcoh took: 1:36:03

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    112.    Elapsed: 0:34:29.
  Batch    80  of    112.    Elapsed: 1:09:03.

  Average training loss: 0.35
  Training epcoh took: 1:36:25

======== Epoch 4 / 4 ========
Training...
  Batch    40  of    112.    Elapsed: 0:34:22.
  Batch    80  of    112.    Elapsed: 1:08:45.

  Average training loss: 0.35
  Training epcoh took: 1:35:57


In [ ]:
output_model = './sentiment.pth'

# save model
def save(bert_classifier, optimizer):
    # save
    torch.save({
        'bert_classifier_state_dict': bert_classifier.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

save(bert_classifier, optimizer)

In [32]:
# to load
checkpoint = torch.load('./sentiment.pth')
bert_classifier.load_state_dict(checkpoint['bert_classifier_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

### Prediction

In [33]:
def predict(model,  prediction_dataloader):
    # Put model in evaluation mode
    model.eval()
    # Tracking variables 
    predictions , true_labels = [], []
    # Predict 
    for batch in prediction_dataloader:
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up predictio
        
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    return predictions

In [ ]:
probs = predict(bert_classifier,test_dataloader)

In [183]:
flat_preds = [item for sublist in preds for item in sublist]
probs=np.argmax(flat_preds, axis=1).flatten()

In [238]:
bert_res = pd.DataFrame(probs, columns = ['Sentiment Score']).set_index(['Sentiment Score'], inplace=False)

In [243]:
df_test[['date','tweet']].reset_index()

,index,date,tweet
0,0,2022-03-10 08:57:26,I can smell the $10k candles $BTC #NFA #DYOR
1,1,2022-03-10 08:52:14,"🚨 🚨 🚨 1,000 #BTC (39,334,386 USD) transferred..."
2,2,2022-03-10 08:46:31,Çoğunluğun düşüncesini merak ediyorum. Benim g...
3,3,2022-03-10 08:44:23,Tatlım benim hadi bakim üzme bak 50 bin tane a...
4,5,2022-03-10 08:42:00,"#Btc cim, rica etsem artık yükselir misin lütf..."
...,...,...,...
5109,8529,2022-03-02 02:45:48,"If I give you $10000 USDT, which crypto would ..."
5110,8535,2022-03-02 02:30:31,"If we look at $VET against $BTC, the chart rem..."
5111,8537,2022-03-02 02:23:34,@BTC_Archive Bad news tbf. PoW is bad for envi...
5112,8538,2022-03-02 02:15:46,"Drop your #NFT, like, follow and Retweet I ha..."


In [242]:
final_test = pd.concat([df_test[['date','tweet']].reset_index(), bert_res], axis=1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [240]:
final_test

,date,tweet
0,2022-03-10 08:57:26,I can smell the $10k candles $BTC #NFA #DYOR
1,2022-03-10 08:52:14,"🚨 🚨 🚨 1,000 #BTC (39,334,386 USD) transferred..."
2,2022-03-10 08:46:31,Çoğunluğun düşüncesini merak ediyorum. Benim g...
3,2022-03-10 08:44:23,Tatlım benim hadi bakim üzme bak 50 bin tane a...
5,2022-03-10 08:42:00,"#Btc cim, rica etsem artık yükselir misin lütf..."
...,...,...
8529,2022-03-02 02:45:48,"If I give you $10000 USDT, which crypto would ..."
8535,2022-03-02 02:30:31,"If we look at $VET against $BTC, the chart rem..."
8537,2022-03-02 02:23:34,@BTC_Archive Bad news tbf. PoW is bad for envi...
8538,2022-03-02 02:15:46,"Drop your #NFT, like, follow and Retweet I ha..."


In [210]:
final_test[final_test['Sentiment Score']==0]

,date,tweet,Sentiment Score
10,2022-03-10 08:25:22,next btc top peak 3rd major crypto cycle histo...,0.0
12,2022-03-10 08:17:50,drop nfts buy 10 today nft eth btc elonmusk ar...,0.0
18,NaN,NaN,0.0
19,2022-03-10 08:01:14,ready central bank absolute control rules regu...,0.0
28,2022-03-10 07:50:25,btc magazin bir galatasarayl olarak yenilsek d...,0.0
...,...,...,...
4942,2022-03-05 12:31:09,btcbullaltbear natureofg btc archive ggreenwal...,0.0
4946,NaN,NaN,0.0
4947,NaN,NaN,0.0
4952,2022-03-05 12:13:04,doge cryptocurrency doge dogecoin btc eth cryp...,0.0


In [ ]:
final_table = 